In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib import cm

from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [ ]:
### BASE MAP SETUP AND GRAVITY DATA

# Setup boundary to focus on the Bay Area
x_bounds=(-123, -121)
y_bounds=(37, 39)

gdf = gpd.read_file("../data/geojson/calif_nev_ncei_grav.geojson")

# Filter areas outside the Bay Area
gdf_subset = gdf[(gdf["latitude"] > y_bounds[0]) &
                 (gdf["latitude"] < y_bounds[1]) &
                 (gdf["longitude"] > x_bounds[0]) &
                 (gdf["longitude"] < x_bounds[1])]

# Tiles can be changed for different layouts, e.g. 'openstreetmap'
m = folium.Map(tiles='CartoDB dark_matter', zoom_start=10, location=[37.8, -122.4])

coord_list = list(zip(gdf_subset.geometry.x, gdf_subset.geometry.y))

In [ ]:
### HEATMAP

# Swap columns for heatmap input
data = [(loc[1], loc[0]) for loc in coord_list]

# Add as neww layer called 'Heat Map'
HeatMap(data).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))

In [ ]:
### MARKERCLUSTER

# Create new cluster object and create new layer 
marker_cluster = MarkerCluster().add_to(folium.FeatureGroup(name='Marker Cluster').add_to(m))

# Place coordinates into the object
for loc in coord_list:
    folium.Marker(location=[loc[1], loc[0]]).add_to(marker_cluster)

In [ ]:
### SIMPLE CIRCLE MARKER

# Create new layer object for circle marker
cmLayer = folium.FeatureGroup(name='Circle Marker')

# To plot data points into the layer object
def plotDot(point):
    folium.CircleMarker(location=[point.geometry.y, point.geometry.x],
                        radius=5,
                        weight=0,
                        popup = point.isostatic_anom, # Mouse click popup, can be HTML or String
                        fill_color='#FF0000').add_to(cmLayer)

# Filter out two columns
threecol = pd.DataFrame(gdf_subset, columns=['geometry', 'isostatic_anom'])
# ... and use apply() to insert each point
threecol.apply(plotDot, axis = 1)

cmLayer.add_to(m)

In [ ]:
# CA COUNTY BORDER

# Please visit http://data.ca.gov to retrieve open datasets for boundaries
# Download CA Country Boundary datasets at https://data.ca.gov/dataset/e212e397-1277-4df3-8c22-40721b095f33/resource/b0007416-a325-4777-9295-368ea6b710e6/download/ca-county-boundaries.zip

choro = folium.FeatureGroup(name='CA Borders')

# Use Chlororpleth
folium.Choropleth(
    geo_data='ca_border.geojson',
#     name='choropleth',
#     data=state_data,
#     columns=['State', 'Unemployment'],
#     key_on='feature.id',
#     fill_color='YlGn',
    line_color='white',
    fill_opacity=0
#     line_opacity=0.2,
#     legend_name='Unemployment Rate (%)'
).add_to(choro)

choro.add_to(m)

In [ ]:

# Attach all layers created to the map
folium.LayerControl().add_to(m)

# Draw map
m

In [ ]:
# Utility - Convert *.shp file to .geojson file with GeoPandas

## Load shp data file
# region = gpd.read_file('CA_OPEN/CA_Counties/CA_Counties_TIGER2016.shp')
## Convert resolution to match the base of gravity data we use as sample
# region['geometry'] = region['geometry'].to_crs(epsg=4326)
## Save as .geojson file using GeoJSON engine to convert
# region.to_file("ca_border.geojson", driver="GeoJSON")